# Camera Calibration

In [7]:
import cv2
import numpy as np
import yaml
import os

class CameraCalibration_and_ArUcoPoseEstimator:
    def __init__(self, yaml_file, config_dict):
        self.yaml_file = yaml_file
        self.camera_matrix = None
        self.dist_coeffs = None
        self.load_calibration()
        self.aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
        self.parameters = cv2.aruco.DetectorParameters_create()
        self.marker_size = config_dict["marker_size"]  # Adjust this based on your marker size
    
    def load_calibration(self):
        """Load camera calibration data from a YAML file."""
        with open(self.yaml_file, 'r') as f:
            data = yaml.safe_load(f)
        
        self.camera_matrix = np.array(data['camera_matrix']['data']).reshape(3, 3)
        self.dist_coeffs = np.array(data['distortion_coefficients']['data'])
        
    def get_camera_matrix(self):
        return self.camera_matrix
    
    def get_dist_coeffs(self):
        return self.dist_coeffs

    def detect_and_estimate_pose(self, frame):
        """Detect ArUco markers and estimate their pose in real time."""
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, self.aruco_dict, parameters=self.parameters)

        if len(corners) > 0:
            # Estimate pose for each detected marker
            for i in range(len(corners)):
                object_points = np.array([
                    [-self.marker_size / 2, self.marker_size / 2, 0],  # top-left
                    [self.marker_size / 2, self.marker_size / 2, 0],   # top-right
                    [self.marker_size / 2, -self.marker_size / 2, 0],  # bottom-right
                    [-self.marker_size / 2, -self.marker_size / 2, 0]  # bottom-left
                ], dtype=np.float32)

                image_points = corners[i].reshape(4, 2)  # For the first detected marker
                # print(image_points)
                ret, rvec, tvec = cv2.solvePnP(object_points, image_points, self.camera_matrix, self.dist_coeffs)

                # Draw the 3D axes on the frame for visualization
                if ret:
                    # Draw the 3D frame axes on the detected marker
                    cv2.drawFrameAxes(frame, self.camera_matrix, self.dist_coeffs, rvec, tvec, 0.1)  # 0.1 is the length of the axes

                    # print(tvec)
                    print("Shape of ids: {} | Shape of tvec: {}".format(ids.shape,tvec.shape))
                    # Print and write the ID and the X, Y, Z coordinates of the marker on the frame
                    if ids is not None and ids[i] is not None:
                        marker_id = ids[i][0] # Shape of ids is (num_markers,1)
                        x, y, z = tvec[:,0]  # Shape is (3,1)
                        
                        # Convert position values to text
                        text = f"ID: {marker_id} X: {x:.2f} Y: {y:.2f} Z: {z:.2f}"
                        
                        # Define position to write the text on the frame (above the marker)
                        position = (int(corners[i][0][0][0]), int(corners[i][0][0][1] - 10))  # Slightly above the top-left corner of the marker
                        
                        # Use cv2.putText to draw the text on the frame
                        cv2.putText(frame, text, position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
        return frame



def main():

    config_dict = {"marker_size": 0.09} # 9cm
    # Path to your calibration file

    # Get the directory where the script is located
    script_dir = os.getcwd()

    # Specify the relative path
    yaml_file = os.path.join(script_dir, "..","calibration_files", "cam1.yaml")

    pose_estimator = CameraCalibration_and_ArUcoPoseEstimator(yaml_file,config_dict)
    
    # Initialize the video capture object outside the class
    cap = cv2.VideoCapture(0)  # Use the correct device ID or path for your webcam
    
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    # Main loop
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture frame.")
            break
        
        # Detect ArUco markers and estimate pose
        frame = pose_estimator.detect_and_estimate_pose(frame)

        # Display the frame with detected markers and 3D axes
        frame_resized = cv2.resize(frame,None,fx=0.5,fy=0.5)
        cv2.imshow("ArUco Marker Detection", frame_resized)

        # Exit on 'Esc' key press
        if cv2.waitKey(1) & 0xFF == 27:
            break
    
    # Release resources
    cap.release()
    cv2.destroyAllWindows()
    return pose_estimator


if __name__ == "__main__":
    pose_estimator = main()


[ WARN:0] global ../modules/videoio/src/cap_gstreamer.cpp (935) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


Shape of ids: (1, 1) | Shape of tvec: (3, 1)
Shape of ids: (1, 1) | Shape of tvec: (3, 1)
Shape of ids: (1, 1) | Shape of tvec: (3, 1)
Shape of ids: (1, 1) | Shape of tvec: (3, 1)
Shape of ids: (1, 1) | Shape of tvec: (3, 1)
Shape of ids: (1, 1) | Shape of tvec: (3, 1)
Shape of ids: (1, 1) | Shape of tvec: (3, 1)
Shape of ids: (1, 1) | Shape of tvec: (3, 1)
Shape of ids: (1, 1) | Shape of tvec: (3, 1)
Shape of ids: (1, 1) | Shape of tvec: (3, 1)
Shape of ids: (1, 1) | Shape of tvec: (3, 1)
Shape of ids: (1, 1) | Shape of tvec: (3, 1)
Shape of ids: (1, 1) | Shape of tvec: (3, 1)
Shape of ids: (1, 1) | Shape of tvec: (3, 1)
Shape of ids: (1, 1) | Shape of tvec: (3, 1)
Shape of ids: (1, 1) | Shape of tvec: (3, 1)
Shape of ids: (1, 1) | Shape of tvec: (3, 1)
Shape of ids: (1, 1) | Shape of tvec: (3, 1)
Shape of ids: (1, 1) | Shape of tvec: (3, 1)
Shape of ids: (1, 1) | Shape of tvec: (3, 1)
Shape of ids: (1, 1) | Shape of tvec: (3, 1)
Shape of ids: (1, 1) | Shape of tvec: (3, 1)
Shape of i

In [7]:
pose_estimator.marker_size

0.09